## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                             input_tensor=input_tensor,
                                             pooling=None, classes=10)

model.summary()
'''Resnet 50 架構'''
model2=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model2.summary()

Using TensorFlow backend.


Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

c:\users\rogerchang\anaconda3\envs\tf-gpu2.0\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## 添加層數

In [2]:
x = model.output

x = Flatten()(x)

#FC
x = Dense(output_dim=500, activation='relu')(x)
x = Dense(output_dim=500, activation='relu')(x)
x = Dense(output_dim=100, activation='relu')(x) 

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

model.summary()

Model深度： 137
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
_______________________________________________________________________________

c:\users\rogerchang\anaconda3\envs\tf-gpu2.0\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=500)`
  
c:\users\rogerchang\anaconda3\envs\tf-gpu2.0\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=500)`
  import sys
c:\users\rogerchang\anaconda3\envs\tf-gpu2.0\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
  
c:\users\rogerchang\anaconda3\envs\tf-gpu2.0\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  # Remove the CWD from sys.path while we load stuff.


## 鎖定特定幾層不要更新權重

In [3]:
#for layer in model.layers[:100]:
    #layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 104s 2ms/step - loss: 1.4204 - accuracy: 0.4746 - val_loss: 0.9887 - val_accuracy: 0.6590
Epoch 2/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.8720 - accuracy: 0.7152 - val_loss: 0.7851 - val_accuracy: 0.7474
Epoch 3/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.6748 - accuracy: 0.7837 - val_loss: 0.6828 - val_accuracy: 0.7834
Epoch 4/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.5389 - accuracy: 0.8259 - val_loss: 0.7140 - val_accuracy: 0.7647
Epoch 5/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.4512 - accuracy: 0.8553 - val_loss: 0.6346 - val_accuracy: 0.7976
Epoch 6/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.3812 - accuracy: 0.8780 - val_loss: 0.5616 - val_accuracy: 0.8165
Epoch 7/100
50000/50000 [==============================] - 93

50000/50000 [==============================] - 93s 2ms/step - loss: 0.0389 - accuracy: 0.9884 - val_loss: 0.8798 - val_accuracy: 0.8320
Epoch 56/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0366 - accuracy: 0.9895 - val_loss: 1.1299 - val_accuracy: 0.8295
Epoch 57/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0345 - accuracy: 0.9890 - val_loss: 1.6434 - val_accuracy: 0.8371
Epoch 58/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0367 - accuracy: 0.9895 - val_loss: 1.2734 - val_accuracy: 0.8287
Epoch 59/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0438 - accuracy: 0.9878 - val_loss: 0.9047 - val_accuracy: 0.8436
Epoch 60/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0338 - accuracy: 0.9904 - val_loss: 2.2153 - val_accuracy: 0.8350
Epoch 61/100
50000/50000 [==============================] - 94s 2ms/step - loss: 0.0392 - accuracy: 0.9888 - val_loss: 